In [3]:
# Importing important packages
import pickle
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from keras import layers

2026-02-16 14:12:05.007293: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-16 14:12:05.026520: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-02-16 14:12:05.693408: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


## Import data

In [4]:
# load signal data
Path = "/home/student/Prathamesh's Project Pre-requisites/CodeSpace/DL/CheckPoint1/"
signal_files = ["/X_TRAIN.pkl","/X_TEST.pkl","/X_VAL.pkl",]

x_train = list()
x_test = list()
x_val = list()

for i in signal_files:
    if i == "/X_TRAIN.pkl":
        with open(Path + "/X_TRAIN.pkl", "rb") as f:
            x_train = pickle.load(f)
    elif i == "/X_TEST.pkl":
        with open(Path + "/X_TEST.pkl", "rb") as f:
            x_test = pickle.load(f)
    elif i == "/X_VAL.pkl":
        with open(Path + "/X_VAL.pkl", "rb") as f:
            x_val = pickle.load(f)

# convert to from np arrays to tensors
x_train = tf.convert_to_tensor(x_train, dtype = tf.float32)
x_test = tf.convert_to_tensor(x_test, dtype = tf.float32)
x_val = tf.convert_to_tensor(x_val, dtype = tf.float32)

In [5]:
# Verify shapes
print(f"The shapes are:\n- x_train is {x_train.shape} \n- x_test is {x_test.shape} \n- x_val is {x_val.shape}")

The shapes are:
- x_train is (12123, 1000, 12) 
- x_test is (1512, 1000, 12) 
- x_val is (1479, 1000, 12)


In [6]:
# load label data
Path = "/home/student/Prathamesh's Project Pre-requisites/CodeSpace/DL/CheckPoint1/"
label_files = ["/Y_TRAIN.pkl","/Y_TEST.pkl","/Y_VAL.pkl",]

for i in label_files:
    if i == "/Y_TRAIN.pkl":
        with open(Path + "/Y_TRAIN.pkl", "rb") as f:
            y_train = pickle.load(f)
    elif i == "/Y_TEST.pkl":
        with open(Path + "/Y_TEST.pkl", "rb") as f:
            y_test = pickle.load(f)
    elif i == "/Y_VAL.pkl":
        with open(Path + "/Y_VAL.pkl", "rb") as f:
            y_val = pickle.load(f)

# process labels
def multihot_encode(pd_series_labels):
    # pandas series to list
    list_labels = pd_series_labels.to_list()

    # convert to ragged vector
    ragged_list_labels = tf.ragged.constant(list_labels)

    # setup string lookup and adapt
    str_lookup_layer = layers.StringLookup(output_mode = "multi_hot", num_oov_indices = 0)
    str_lookup_layer.adapt(ragged_list_labels) 
    multi_hot_labels = str_lookup_layer(ragged_list_labels)

    return multi_hot_labels

# appyl multi hot encoding
y_train = tf.cast(multihot_encode(y_train),dtype = tf.float32)
y_test = tf.cast(multihot_encode(y_test),dtype = tf.float32)
y_val = tf.cast(multihot_encode(y_val),dtype = tf.float32)

In [7]:
# Verify shapes
print(f"The shapes are:\n- y_train is {y_train.shape} \n- y_test is {y_test.shape} \n- y_val is {y_val.shape}")

The shapes are:
- y_train is (12123, 6) 
- y_test is (1512, 6) 
- y_val is (1479, 6)


## Random Over-sampling of Training Set

In [50]:
# Define algorithm

def find_label_imbalance_ratio(Y):
    '''
    Args:
    X:  preferebally, 3D tensor of time-series data
    Y: preferebally, 2D tensor of multi-hot encoded labels

    Returns:
    Imbalance ratio for each label

    Pseudo:
    - Consider, instance x has label with different classes till q (1<= l <= q); in our case it is 7
    - for l=1 till q: find the number of occurances of each
    - Define Nmaj as label with maximum instances
    - for l=1 till q: finf IR_Lb = (Nmaj/Nl) 
    '''

    Y_DF = pd.DataFrame(Y)
    label_counts = tf.reduce_sum(Y_DF, axis = 0)
    max_label = tf.reduce_max(label_counts)
    IR_LB = max_label/label_counts

    return label_counts, IR_LB

def partial_random_oversampling(X, Y, r = 3.0):
    '''
    Args:
    X:  preferebally, 3D tensor of time-series data
    Y: preferebally, 2D tensor of multi-hot encoded labels

    Returns:
    Rescaled features and targets as per the imbalance threshold

    Pseudo:
    - set s_ij as 1; considering all classlabels and instances
    - for each class label l till q: find the Imbalance ratio of l
    - if it is > than imbalance threshold: randomise the instances relevent with that class label
    - till the imbalance ratio is greater than imbalance threshold:
        - get the label from one of the randomised instance
        - create a new instance
        - set 1 for that class label and keep zero for the others
    '''

    label_counts, IR_LB = find_label_imbalance_ratio(Y)
    X_resample = list()
    Y_resample = list()
    for i in range(len(IR_LB)):
        if IR_LB[i] > r:
            for j in range(len(X)):
                if Y[j, i] == 1.0 and sum(Y[j, ]) == 1.0:
                    X_resample.append(X[j-1:j])
                    Y_resample.append(Y[j,])
    
    return X_resample, Y_resample


In [51]:
x_sam, y_sam = partial_random_oversampling(x_train, y_train, r= 3.0)